## The one that works

In [4]:
from __future__ import print_function

%matplotlib notebook 
import os
print(os.getcwd())

import tensorflow as tf
import numpy as np

from os.path import join
from helper_functions import *
from visualization import Visualization
from sklearn.model_selection import train_test_split

EPOCHS = 200
LEARNING_RATE = 1e-4

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
    
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
    
class Number_Data():
    def __init__(self, data, filter_type='original'):
        self._index_in_epoch = 0
        self._epochs_completed = 0
        self.viz = Visualization(epochs=EPOCHS)
        self._x_data, self._y_data = collect_images(data, filter_type)                
        target = np.zeros((len(self._y_data), 10))
        target[np.arange(len(self._y_data)),self._y_data] = 1
        self._y_data = target
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self._x_data,self._y_data,random_state=42)
        self._num_examples = len(self.x_train)
        self.train_data_count = len(self.x_train)
        self.test_data_count = len(self.x_test)
        
        # Flatten training images.
        n_samples = len(self.x_train)
        self.x_train = self.x_train.reshape((n_samples,-1))
        n_samples = len(self.x_test)
        self.x_test = self.x_test.reshape((n_samples,-1))        
        
        # Get flattened test images.
        self.test_images = self.get_test_images()
        self.test_labels = self.y_test
        
        sess = tf.InteractiveSession()
        
        # Initialize input and target tensors.
        with tf.name_scope('input'):
            self.x = tf.placeholder(tf.float32, shape=[None, 784], name='x-input')
            print(self.x)
            self.y_ = tf.placeholder(tf.float32, shape=[None, 10], name='y-input')

        # Initialize weight and bias tensors.
        with tf.name_scope('convolutional_layer_1'):
            W_conv1 = weight_variable([5, 5, 1, 32])
            b_conv1 = bias_variable([32])
        
        with tf.name_scope('input_reshape'):
            x_image = tf.reshape(self.x, [-1,28,28,1])
            tf.summary.image('input', x_image, 10)
            
        with tf.name_scope('convolutional_layer1'):
            h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
            
        with tf.name_scope('pooling_layer_1'):
            h_pool1 = max_pool_2x2(h_conv1)
        
        with tf.name_scope('convolutional_layer_2'):
            W_conv2 = weight_variable([5, 5, 32, 64])
            b_conv2 = bias_variable([64])        
            h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        
        with tf.name_scope('pooling_layer_2'):
            h_pool2 = max_pool_2x2(h_conv2)
            
        with tf.name_scope('fully_connected_layer_1'):
            W_fc1 = weight_variable([7 * 7 * 64, 512])
            b_fc1 = bias_variable([512])
            h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
            h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        
        with tf.name_scope('dropout'):
            self.keep_prob = tf.placeholder(tf.float32)
            tf.summary.scalar('dropout_keep_probability', self.keep_prob)
            h_fc1_drop = tf.nn.dropout(h_fc1, self.keep_prob)        

        with tf.name_scope('fully_connected_layer_2'):
            W_fc2 = weight_variable([512, 10])
            b_fc2 = bias_variable([10])
            self.y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

        with tf.name_scope('cross_entropy'):
            # The raw formulation of cross-entropy,
            #
            # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
            #                               reduction_indices=[1]))
            #
            # can be numerically unstable.
            #
            # So here we use tf.nn.softmax_cross_entropy_with_logits on the
            # raw outputs of the nn_layer above, and then average across
            # the batch.
            
            diff = tf.nn.softmax_cross_entropy_with_logits(self.y_conv, self.y_)
            with tf.name_scope('total'):
                self.cross_entropy = tf.reduce_mean(diff)

        tf.summary.scalar('cross_entropy', self.cross_entropy)

        with tf.name_scope('train'):
            self.train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(self.cross_entropy)

        with tf.name_scope('accuracy'):
            with tf.name_scope('correct_prediction'):
                correct_prediction = tf.equal(tf.argmax(self.y_conv, 1), tf.argmax(self.y_, 1))
            with tf.name_scope('accuracy'):
                self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', self.accuracy)
        
        # Merge all the summaries and write them out to /logs
        dir_path = os.path.abspath('')
        log_dir=join(dir_path,'logs')        
        self.merged = tf.summary.merge_all()
        self.train_writer = tf.summary.FileWriter(join(log_dir,'train'),
                                      sess.graph)
        self.test_writer = tf.summary.FileWriter(join(log_dir,'test'))
        tf.global_variables_initializer().run()
            
    def next_batch(self, batch_size=100, shuffle=True):
        start = self._index_in_epoch
        # Shuffle for the first epoch
        if self._epochs_completed == 0 and start == 0 and shuffle:
            perm0 = np.arange(self._num_examples)
            np.random.shuffle(perm0)
            self._images = self.x_train[perm0]
            self._labels = self.y_train[perm0]
        # Go to the next epoch
        if start + batch_size > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Get the rest examples in this epoch
            rest_num_examples = self._num_examples - start
            images_rest_part = self.x_train[start:self._num_examples]
            labels_rest_part = self.y_train[start:self._num_examples]
            # Shuffle the data
            if shuffle:
                perm = np.arange(self._num_examples)
                np.random.shuffle(perm)
                self._images = self._images[perm]
                self._labels = self._labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            images_new_part = self._images[start:end]
            labels_new_part = self._labels[start:end]
            return np.concatenate((images_rest_part, images_new_part), axis=0) , np.concatenate((labels_rest_part, labels_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch            
            return self._images[start:end], self._labels[start:end]
        
    def __call__(self,feed_list):        
        """ Classify image. """
        
        feed_dict = {self.x : feed_list[0], self.y_: feed_list[1], self.keep_prob:feed_list[2]}
#         classification = tf.run(self.y_, feed_dict)
#         print(classification)
        sess = tf.get_default_session()
        classification = sess.run(self.y_conv, feed_dict)
        print(classification)
        train_accuracy = self.accuracy.eval(feed_dict=feed_dict)
        print("test accuracy %g"%train_accuracy)

    def get_test_images(self):
        images = self.x_test
        n_samples = len(images)
        return images.reshape((n_samples, -1))

    def train_network(self):
        sess = tf.get_default_session()
        
        for epoch in range(EPOCHS):
            batch = self.next_batch(100)    
            summary, _, acc, cross_entropy = sess.run([self.merged, self.train_step, self.accuracy, self.cross_entropy], feed_dict={self.x : batch[0], self.y_ : batch[1], self.keep_prob: 0.5})
            
            if epoch % 100 == 99: # Record train set summaries, and test                
                run_metadata = tf.RunMetadata()
                self.train_writer.add_run_metadata(run_metadata, 'step%03d' % epoch)

                # Add training summary
                self.train_writer.add_summary(summary, epoch)
                                
                summary, test_accuracy, cross_entropy = sess.run([self.merged, self.accuracy, self.cross_entropy],feed_dict={self.x : self.test_images, self.y_ : self.test_labels, self.keep_prob: 1.0})
                print("step %d, test accuracy %g"%(epoch, test_accuracy))
                self.viz(epoch, test=test_accuracy, cross_entropy=cross_entropy)
                self.test_writer.add_summary(summary, epoch)
            
            # Visualize training progress every 25 steps.
            if epoch % 25 == 0:                            
                print("step %d, training accuracy %g"%(epoch, acc))
                self.viz(epoch, train=acc, cross_entropy=cross_entropy)
          
        self.train_writer.close()
        self.test_writer.close()

# Load dataset.
data = pickle.load(open('data3_4.p', 'rb'))

number_data = Number_Data(data, 'original')
number_data.train_network()

/Users/justinshenk/Projects/tensorflow/digits-classifier/src


ImportError: No module named 'visualization'

In [ ]:
# Check individual image classification

for i in range(10):
    image = data['original'][-i].reshape((-1,784))
    target_number = data['targets'][-i]
    target = np.zeros((1, 10))
    target[np.arange(1),target_number] = 1    
    print(target,target_number)
    print(image.shape)    
    
    feed_list = [image, target, 1.0]
    number_data(feed_list)
    plt.imshow(image.reshape((28,28)),cmap=plt.cm.gray)
    plt.show()